In [2]:
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math

import requests
from bs4 import BeautifulSoup
import wget
import os

#from scrape import *
#from get_metadata import *

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from time import sleep
import json
import datetime
import os


def format_day(date):
    day = '0' + str(date.day) if len(str(date.day)) == 1 else str(date.day)
    month = '0' + str(date.month) if len(str(date.month)) == 1 else str(date.month)
    year = str(date.year)
    return '-'.join([year, month, day])


def form_url(user, since, until):
    p1 = 'https://twitter.com/search?f=tweets&vertical=default&q=from%3A'
    p2 = user + '%20since%3A' + since + '%20until%3A' + until + 'include%3Aretweets&src=typd'
    return p1 + p2


def increment_day(date, i):
    return date + datetime.timedelta(days=i)


def get_tweet_ids(user, start, end, output_path=None):
    # edit these three variables
    #user = 'elonmusk'
    #start = datetime.datetime(2019, 1, 16)  # year, month, day
    #end = datetime.datetime(2019, 1, 17)  # year, month, day

    if not output_path:
        output_path = '/Users/nesara/Documents/aim/git/articles/twitter_analysis/data/'

    user_dir_path = output_path + user
    if not os.path.exists(user_dir_path):
        os.makedirs(user_dir_path)
    user_dir_path = user_dir_path + "/"

    # only edit these if you're having problems
    delay = 1  # time to wait on each page load before reading the page
    driver = webdriver.Safari()  # options are Chrome() Firefox() Safari()


    # don't mess with this stuff
    twitter_ids_filename = user_dir_path + 'all_ids.json'
    days = (end - start).days + 1
    id_selector = '.time a.tweet-timestamp'
    tweet_selector = 'li.js-stream-item'
    user = user.lower()
    ids = []

    for day in range(days):
        d1 = format_day(increment_day(start, 0))
        d2 = format_day(increment_day(start, 1))
        url = form_url(user, d1, d2)
        #print(url)
        #print(d1)
        driver.get(url)
        sleep(delay)

        try:
            found_tweets = driver.find_elements_by_css_selector(tweet_selector)
            increment = 10

            while len(found_tweets) >= increment:
                #print('scrolling down to load more tweets')
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                sleep(delay)
                found_tweets = driver.find_elements_by_css_selector(tweet_selector)
                increment += 10

            print('{} tweets found, {} total'.format(len(found_tweets), len(ids)))

            for tweet in found_tweets:
                try:
                    id = tweet.find_element_by_css_selector(id_selector).get_attribute('href').split('/')[-1]
                    ids.append(id)
                except StaleElementReferenceException as e:
                    print('lost element reference', tweet)

        except NoSuchElementException:
            print('no tweets on this day')

        start = increment_day(start, 1)
    sleep(delay)
    driver.close()
    driver.quit()

    try:
        with open(twitter_ids_filename) as f:
            all_ids = ids + json.load(f)
            data_to_write = list(set(all_ids))
            print('tweets found on this scrape: ', len(ids))
            print('total tweet count: ', len(data_to_write))
    except FileNotFoundError:
        with open(twitter_ids_filename, 'w') as f:
            all_ids = ids
            data_to_write = list(set(all_ids))
            print('tweets found on this scrape: ', len(ids))
            print('total tweet count: ', len(data_to_write))

    with open(twitter_ids_filename, 'w') as outfile:
        json.dump(data_to_write, outfile)

    print('all done here')
    #driver.close()

In [9]:
import tweepy
import json
import math
import glob
import csv
import zipfile
import zlib
from tweepy import TweepError
from time import sleep
import os


def is_retweet(entry):
    return 'retweeted_status' in entry.keys()


def get_source(entry):
    if '<' in entry["source"]:
        return entry["source"].split('>')[1].split('<')[0]
    else:
        return entry["source"]

########## API Keys ##############
with open('api_keys.json') as f:
    keys = json.load(f)

auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth)


def get_metadata(user, output_path=None):
    # CHANGE THIS TO THE USER YOU WANT
    #user = 'elonmusk'
    user = user.lower()
    if not output_path:
        output_path = '/Users/nesara/Documents/aim/git/articles/twitter_analysis/data/'

    user_dir_path = output_path + user
    if not os.path.exists(user_dir_path):
        os.makedirs(user_dir_path)
    user_dir_path = user_dir_path + "/"

    output_file = user_dir_path + '{}.json'.format(user)
    output_file_short = user_dir_path + '{}_short.json'.format(user)
    #compression = zipfile.ZIP_DEFLATED
    profile_info_file = user_dir_path + '{}_profile.json'.format(user)
    output_csv_file = user_dir_path + '{}.csv'.format(user)

    with open(user_dir_path + 'all_ids.json') as f:
        ids = json.load(f)

    print('total ids: {}'.format(len(ids)))

    all_data = []
    start = 0
    end = 100
    limit = len(ids)
    i = math.ceil(limit / 100)

    for go in range(i):
        print('currently getting {} - {}'.format(start, end))
        sleep(6)  # needed to prevent hitting API rate limit
        id_batch = ids[start:end]
        start += 100
        end += 100
        tweets = api.statuses_lookup(id_batch)
        for tweet in tweets:
            all_data.append(dict(tweet._json))

    print('metadata collection complete')
    print('creating master json file')
    with open(output_file, 'w') as outfile:
        json.dump(all_data, outfile)

    """
    print('creating ziped master json file')
    zf = zipfile.ZipFile('{}.zip'.format(user), mode='w')
    zf.write(output_file, compress_type=compression)
    zf.close()
    """

    ###### Get user profile info #########
    results = []
    with open(output_file) as json_data:
        data = json.load(json_data)
        for entry in data:
            t = {
                "name": entry["user"]["name"],
                "screen_name": entry["user"]["screen_name"],
                "followers_count": entry["user"]["followers_count"],
                "friends_count": entry["user"]["friends_count"],
                "listed_count": entry["user"]["listed_count"],
                "favourites_count": entry["user"]["favourites_count"],
                "created_at": entry["user"]["created_at"],
                "statuses_count": entry["user"]["statuses_count"]
            }
            results.append(t)

    print('creating profile info file')
    with open(profile_info_file, 'w') as outfile:
        json.dump(results, outfile)

    ######### Get tweet info ###########
    results = []
    with open(output_file) as json_data:
        data = json.load(json_data)
        for entry in data:
            t = {
                "created_at": entry["created_at"],
                "text": entry["text"],
                "in_reply_to_screen_name": entry["in_reply_to_screen_name"],
                "retweet_count": entry["retweet_count"],
                "favorite_count": entry["favorite_count"],
                "source": get_source(entry),
                "is_retweet": is_retweet(entry),
                "geo": entry["geo"],
                "coordinates": entry["coordinates"],
                "place": entry["place"]
            }
            results.append(t)

    print('creating minimized json master file')
    with open(output_file_short, 'w') as outfile:
        json.dump(results, outfile)

    with open(output_file_short) as master_file:
        data = json.load(master_file)
        fields = ["favorite_count", "source", "text", "in_reply_to_screen_name",
                  "is_retweet", "created_at", "retweet_count", "geo", "coordinates", "place"]
        print('creating CSV version of minimized json master file')
        f = csv.writer(open(output_csv_file, 'w'))
        f.writerow(fields)
        for x in data:
            f.writerow([x["favorite_count"], x["source"], x["text"], x["in_reply_to_screen_name"],
                        x["is_retweet"], x["created_at"], x["retweet_count"], x["geo"], x["coordinates"], x["place"]])


#### Get top twitter accounts

In [3]:
base_url = "https://en.wikipedia.org/wiki/List_of_most-followed_Twitter_accounts"
res = requests.get(base_url)
soup = BeautifulSoup(res.content, 'lxml')

In [4]:
users = []
user_list = soup.findAll("td")[2::7]
for i,u in enumerate(user_list[:-1]):
    users.append(u.text[1:-1])
users

['katyperry',
 'justinbieber',
 'BarackObama',
 'rihanna',
 'taylorswift13',
 'ladygaga',
 'TheEllenShow',
 'Cristiano',
 'YouTube',
 'jtimberlake',
 'ArianaGrande',
 'KimKardashian',
 'realDonaldTrump',
 'selenagomez',
 'britneyspears',
 'twitter',
 'cnnbrk',
 'shakira',
 'jimmyfallon',
 'BillGates',
 'narendramodi',
 'JLo',
 'nytimes',
 'neymarjr',
 'BrunoMars',
 'KingJames',
 'Oprah',
 'MileyCyrus',
 'CNN',
 'NiallOfficial',
 'BBCBreaking',
 'Drake',
 'iamsrk',
 'instagram',
 'SrBachchan',
 'BeingSalmanKhan',
 'SportsCenter',
 'KevinHart4real',
 'LilTunechi',
 'espn',
 'wizkhalifa',
 'Harry_Styles',
 'Louis_Tomlinson',
 'LiamPayne',
 'Pink',
 'realmadrid',
 'onedirection',
 'NASA',
 'aliciakeys',
 'chrisbrown']

In [5]:
len(users)

50

In [12]:
type(users)

list

In [15]:
for x in users:
    print('"'+x+'",', end=' ')

"katyperry", "justinbieber", "BarackObama", "rihanna", "taylorswift13", "ladygaga", "TheEllenShow", "Cristiano", "YouTube", "jtimberlake", "ArianaGrande", "KimKardashian", "realDonaldTrump", "selenagomez", "britneyspears", "twitter", "cnnbrk", "shakira", "jimmyfallon", "BillGates", "narendramodi", "JLo", "nytimes", "neymarjr", "BrunoMars", "KingJames", "Oprah", "MileyCyrus", "CNN", "NiallOfficial", "BBCBreaking", "Drake", "iamsrk", "instagram", "SrBachchan", "BeingSalmanKhan", "SportsCenter", "KevinHart4real", "LilTunechi", "espn", "wizkhalifa", "Harry_Styles", "Louis_Tomlinson", "LiamPayne", "Pink", "realmadrid", "onedirection", "NASA", "aliciakeys", "chrisbrown", 

#### Get tweet ids

In [18]:
# Get all tweet ids of user from start_date to end_date
output_path = '/Users/nesara/Documents/aim/git/articles/twitter_analysis/data/'
start_date = datetime.datetime(2019, 2, 21)  # year, month, day
end_date = datetime.datetime(2019, 2, 22)  # year, month, day

for user in users[:5]:
    print("Getting ids of user " + user + "...")
    get_tweet_ids(user, start_date, end_date, output_path)

    # Get metadata from tweet ids
    print("Getting metadata of user " + user + "...")
    get_metadata(user, output_path)

Getting ids of user katyperry...
0 tweets found, 0 total
0 tweets found, 0 total
tweets found on this scrape:  0
total tweet count:  3
all done here
Getting metadata of user katyperry...
total ids: 3
currently getting 0 - 100
metadata collection complete
creating master json file
creating profile info file
creating minimized json master file
creating CSV version of minimized json master file
Getting ids of user justinbieber...
0 tweets found, 0 total
0 tweets found, 0 total
tweets found on this scrape:  0
total tweet count:  0
all done here
Getting metadata of user justinbieber...
total ids: 0
metadata collection complete
creating master json file
creating profile info file
creating minimized json master file
creating CSV version of minimized json master file
Getting ids of user BarackObama...
2 tweets found, 0 total
0 tweets found, 2 total
tweets found on this scrape:  2
total tweet count:  2
all done here
Getting metadata of user BarackObama...
total ids: 2
currently getting 0 - 100


### Load tweets from top users

In [4]:
user_list_dir = "/Users/nesara/Documents/aim/git/articles/twitter_analysis/data_sample/"

df = pd.DataFrame()
for user in os.listdir(user_list_dir):
    if not user.startswith("."):
        user_lower = user.lower()
        user_csv_file_path = user_list_dir + user + "/" + user_lower + ".csv"
        if os.path.exists(user_csv_file_path):
            df_user = pd.read_csv(user_csv_file_path)
            df_user["user"] = user_lower
            del df_user["geo"]
            df = df.append(df_user)

In [5]:
df["created_at"] = pd.to_datetime(df["created_at"])
df

,favorite_count,source,text,in_reply_to_screen_name,is_retweet,created_at,retweet_count,coordinates,place,user
0,17839,Twitter Web Client,T 3057 - सूर्य के उत्‍तरायण होने पर मौसम बदलने...,NaN,False,2019-01-13 02:27:37,1792,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan
1,45,Twitter for iPhone,@BeejalBhatt Guddi,BeejalBhatt,False,2019-01-25 07:40:11,19,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan
2,35,Twitter for iPhone,@prashantkawadia @TeamSrbachchan @juniorbachch...,prashantkawadia,False,2019-01-25 10:28:27,14,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan
3,5593,Twitter Web Client,:))) https://t.co/sx32gA7vJu,NaN,False,2019-01-27 19:34:39,236,NaN,NaN,srbachchan
4,35,Twitter Web Client,@JoNOoON_ thank you .. {}{}{},JoNOoON_,False,2019-01-19 20:56:40,9,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan
5,16846,Twitter for iPhone,"T 3056 - ‘It's not in destiny nor it's fate, h...",NaN,False,2019-01-12 06:29:21,1152,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan
6,32,Twitter for iPhone,@prashantkawadia @TeamSrbachchan @juniorbachch...,prashantkawadia,False,2019-01-25 10:33:25,17,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan
7,13133,Twitter Web Client,T 3057 - .. at the Gurudwara for 'ardaas' .. G...,NaN,False,2019-01-13 18:40:02,704,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan
8,20385,Twitter Web Client,T 3062 - BADHAI BADHAI BADHAI .. Indian Cricke...,NaN,False,2019-01-18 19:01:35,1213,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan
9,31,Twitter for iPhone,@rb_Prajapati90 Thank you,rb_Prajapati90,False,2019-01-25 07:37:00,11,NaN,"{'id': '7929cea6bd5b32bd', 'url': 'https://api...",srbachchan


In [6]:
df.groupby("user").count()["text"]

user
aliciakeys           37
arianagrande        309
barackobama           4
bbcbreaking          84
beingsalmankhan      10
billgates            22
britneyspears         5
brunomars             6
cnn                2117
cristiano            20
ddlovato              2
drake                 1
espn                439
harry_styles          1
iamsrk                3
instagram            44
jimmyfallon          63
jlo                  18
jtimberlake           5
justinbieber          2
katyperry            13
kevinhart4real       90
kimkardashian        69
kingjames            23
ladygaga             11
liampayne             8
liltunechi            8
louis_tomlinson      16
mileycyrus           54
narendramodi        318
nasa                 27
neymarjr             18
niallofficial        63
nytimes            1877
oprah                 9
pink                  4
realdonaldtrump     232
realmadrid          262
rihanna               8
selenagomez           1
shakira               6
sportscente

In [7]:
user = "arianagrande"
dfu = df[df["user"]==user][["text", "source", "favorite_count", "retweet_count", "is_retweet"]]
for i in range(len(dfu)):
    if not dfu.iloc[i]["is_retweet"]:
        print(dfu.iloc[i]["text"])

@buterawoah @alrightavenue @arianalgbtq i don’t. i’m saying thank u.
eternally* cool
the way ..... it shiiiiiiine i know ...... u seen it
kno what https://t.co/k0FFskYJtH
@notearssamantha SKSJSKSJS the light said brb
i really 😭😭🖤 https://t.co/VmbmYNcytw
@remcabello @sorryidipped @bieberftswift breakup w ur gf.... on a less deep note
@remstyIes @minajcum see what y’all do to me
@arigIows 🖤
🖤🌪 2️⃣ 💎 https://t.co/Idf38u2oud
if she allows me https://t.co/CwoDlbkhQ5
@ghostindeIuxe @vibinrem @deIuxeverytimes @notearssamantha slightly different now but you’ll see it soon enough. love u.
love u 🖤💎💧🌬🌫🌪3️⃣
💍💍💍 of 7 https://t.co/LraKv7wtGT
@raysboca うたいましょう！ let’s sing :)
again, not lyrics... simply how i feel ab u all
@successfvI @lookgoodright yea man i like peeled my skin off my face and the dw mask was underneath. it was ambitio… https://t.co/o9ILYmaOM0
i believe so 🌪🌫 https://t.co/Up4pIhRCv3
@goawaynina @everydaysus yea it was really bad im deleting it
@everytimebeth @breathinsarah that’s a 

In [8]:
df.groupby("source").count()["text"]

source
Google                     4
Instagram                 65
Periscope                 34
SnappyTV.com              10
SocialFlow              3682
Spredfast app            709
Sprinklr                  47
Sprout Social              1
The New York Times         1
TweetDeck                204
Tweetbot for iΟS           1
Twitlonger                 1
Twitter Ads               22
Twitter Ads Composer       7
Twitter Media Studio     602
Twitter Web App           34
Twitter Web Client       737
Twitter for Android        6
Twitter for iPad           1
Twitter for iPhone      1092
Name: text, dtype: int64